In [0]:
import sys
from analytical import *

# The model

We are trying to infer 4 key variables
* Prevalence $p$.
* Testing rate $f$ of mild or asympromatic carriers.
* Total infected population $R_p$.

We are given as data:
* The number of deaths $D$ that tested positive with COVID.
* The number of recoveries $R$ that tested positive with COVID.
* The total population $P$.

We have to make assumptions about:
* The Case Fatality Rate (CFR) due to COVID $\text{CFR}_c$.
* The CFR due to other reasons with similar symptoms as COVID $\text{CFR}_o$. We set it to $0.01/100$ (Annual mortality of 1% * 1/12 months * ~1/10 serious conditions mau be confused.)
* The increase in risk to be infected with COVID if in a serious condition $\mu$. We usually set $\mu=10$, namely being in a serious medical condition leading to death (for other reasons) also exposes a patient to the equavalent of x10 to catch COVID -- due to hospital infection rates.

The system of equations we need to solve are:
* The hospital prevalence $p_h$:
\begin{equation}
h_p = 1 - (1 - p)^{\mu}
\end{equation}
* The actual deaths due to COVID. All deaths minus the ones due to other causes that still tested positive for COVID:
\begin{equation}
D_a = \max{(D - p_h \cdot \text{CFR}_o \cdot P, 1)}
\end{equation}
* The total infections in the population is actual infected and deaths due to Covid
\begin{equation}
p = (R_p + D) / P
\end{equation}
* The observed recoveries are a sample of the actual one based on testing rates.
\begin{equation}
R = f \cdot R_p
\end{equation}
* The definition of the Case Fatality rate for COVID is:
\begin{equation}
\text{CFR}_c = \frac{D_a}{D_a + R_p}
\end{equation}

This system of equaltions is non-linear but we can solve it numericaly using iterative methods.







# Results for different $\text{CFR}_c$ 

In [0]:
# Since we measure prevalence based on outcomes, the figures lag by about 20 days.
CFR_covid = 0.01 # CFR medium high: 1%
hospital_infection_mult = 10.0
make_table(populations, CFR_covid, hospital_infection_mult, flx=sys.stdout)

Assumptions: COVID CFR:  1.00% In Hospital factor: x10.0
Country           Prev      CFR  Testing   Comorb.    Infected
--------------------------------------------------------------
Japan            0.00%    1.00%    6.22%    9.09%        3,776
USA              0.01%    1.00%    0.63%    9.09%       46,485
Germany          0.01%    1.00%    4.10%    9.09%       11,059
Italy            0.92%    1.00%    1.36%    8.76%      548,407
Spain            0.43%    1.00%    1.69%    8.93%      198,779
Belgium          0.07%    1.00%    5.07%    9.06%        7,914
Switzerland      0.13%    1.00%    1.23%    9.04%       10,614
Iran             0.20%    1.00%    5.14%    9.02%      163,051
Korea, South     0.02%    1.00%   31.72%    9.08%        9,980
United Kingdom   0.05%    1.00%    0.45%    9.07%       30,125
Netherlands      0.11%    1.00%    0.01%    9.05%       19,159
France           0.12%    1.00%    2.84%    9.05%       77,359


In [0]:
# Since we measure prevalence based on outcomes, the figures lag by about 20 days.
CFR_covid = 0.001 # CFR low: 0.1%
hospital_infection_mult = 10.0
make_table(populations, CFR_covid, hospital_infection_mult, flx=sys.stdout)

Assumptions: COVID CFR:  0.10% In Hospital factor: x10.0
Country           Prev      CFR  Testing   Comorb.    Infected
--------------------------------------------------------------
Japan            0.02%    0.10%    1.12%   49.98%       20,965
USA              0.08%    0.10%    0.11%   49.91%      258,443
Germany          0.07%    0.10%    0.74%   49.92%       61,479
Italy            5.64%    0.10%    0.22%   43.84%    3,406,502
Spain            2.50%    0.10%    0.29%   47.22%    1,162,541
Belgium          0.39%    0.10%    0.91%   49.56%       44,296
Switzerland      0.70%    0.10%    0.22%   49.22%       59,807
Iran             1.14%    0.10%    0.90%   48.72%      927,409
Korea, South     0.11%    0.10%    5.70%   49.88%       55,523
United Kingdom   0.25%    0.10%    0.08%   49.72%      168,119
Netherlands      0.63%    0.10%    0.00%   49.29%      107,789
France           0.65%    0.10%    0.51%   49.27%      435,423


In [0]:
# Since we measure prevalence based on outcomes, the figures lag by about 20 days.
CFR_covid_low = 0.0001 # CFR very low: 0.01%
hospital_infection_mult = 10.0
make_table(populations, CFR_covid_low, hospital_infection_mult, flx=sys.stdout)

Assumptions: COVID CFR:  0.01% In Hospital factor: x10.0
Country           Prev      CFR  Testing   Comorb.    Infected
--------------------------------------------------------------
Japan            0.03%    0.01%    0.62%   90.90%       38,186
USA              0.14%    0.01%    0.06%   90.86%      472,266
Germany          0.14%    0.01%    0.40%   90.86%      112,317
Italy           16.65%    0.01%    0.07%   83.43%    10,066,389
Spain            5.30%    0.01%    0.14%   88.79%    2,471,375
Belgium          0.72%    0.01%    0.49%   90.64%       82,301
Switzerland      1.32%    0.01%    0.12%   90.41%      113,058
Iran             2.22%    0.01%    0.47%   90.06%    1,799,738
Korea, South     0.20%    0.01%    3.12%   90.84%      101,616
United Kingdom   0.47%    0.01%    0.04%   90.73%      310,069
Netherlands      1.18%    0.01%    0.00%   90.46%      202,948
France           1.23%    0.01%    0.27%   90.44%      820,873


# Projection forward

Since the data used to estimate prevalence relates to outcomes, we know that the estimation lags behind by the amount of typical time it takes to have an outcome. From studies we consider this to be about 20 days. We therefore build a projection of the prevalence based on a simple model:
* The increase in prevalence follows the difference equation. Its solution is a logistic curve, and 0.6 represents 60% of the population being infected (after which herd immunity kicks in).
\begin{equation}
dp = (p \cdot r - p) \cdot (0.6 - p)
\end{equation}
* The rate of growth $r$ is computed based on the 3 previous days of growth.
* We assume a low CFR (0.01%).

The resulting projections are:

![COVID prevalence projections for key countries](https://github.com/gdanezis/COVID-Prevalence/blob/master/figures/All-prev.png?raw=1)

The inflence of $\text{CFR}_c$ is not dramatic, but only moves prevance rates by 2 weeks. For the United Kingdom the plot a range of $\text{CFR}_c$ in $[1\%-0.01\%]$ 

![COVID prevalence projections for key countries](https://github.com/gdanezis/COVID-Prevalence/blob/master/figures/United%20Kingdom-prev.png?raw=1)